In [1]:
from project.utils import get_pandas
import pandas as pd
import os

In [130]:
#profeel = get_pandas('project/input/stock/building_stock_profeel_vent.csv', lambda x: pd.read_csv(x, index_col=[0, 1]).squeeze())
profeel = get_pandas('project/input/stock/building_stock_profeel_vent.csv', lambda x: pd.read_csv(x, index_col=[1]).squeeze())
sdes = get_pandas('project/input/stock/building_stock_sdes2018_aggregated.csv', lambda x: pd.read_csv(x, index_col=[0, 1, 2]).squeeze())

profeel = profeel.rename(columns={'calculated_epc':'Energy performance'})
profeel['Heating energy'] = profeel['Heating system'].str.split('-').str[0]
profeel = profeel.set_index(['Heating energy', 'Energy performance'], append=True)

In [134]:
merged = profeel.merge(sdes, how='inner', on=['Housing type', 'Energy performance', 'Heating energy'])
# Adding a row specific index level
r_position = pd.Series(range(0,merged.shape[0]), name='r_position')
merged = merged.set_index(r_position, append=True)

# Weight of each lines in terms of sdes's sizes (y) among a profeel archetype uniquely define by its class label
merged = merged.set_index('Class', append=True)
merged['Stock buildings_y'] = merged['Stock buildings_y'].T / merged.groupby(level='Class')['Stock buildings_y'].sum()

# Multiplying the weights to the profeel's stocks (x)
merged['Stock buildings'] = (merged['Stock buildings_y'].T * merged['Stock buildings_x']).T
cleaned = merged.drop(labels=['Stock buildings_x', 'Stock buildings_y'], axis=1)
cleaned

Heating system  \
Housing type  Energy performance Heating energy r_position Class                                   
Single-family F                  Wood fuel      0          MI-1        Wood fuel-Standard boiler   
                                                1          MI-1        Wood fuel-Standard boiler   
                                                2          MI-1        Wood fuel-Standard boiler   
                                                3          MI-1        Wood fuel-Standard boiler   
                                                4          MI-1        Wood fuel-Standard boiler   
...                                                                                          ...   
Multi-family  B                  Electricity    4675       LC24   Electricity-Performance boiler   
                                                4676       LC24   Electricity-Performance boiler   
                                                4677       LC24   Electricity-Performance boiler   
                                                4678       LC24   Electricity-Performance boiler   
                                                4679       LC24   Electricity-Performance boiler   

                                                                  Wall  Floor  \
Housing type  Energy performance Heating energy r_position Class                
Single-family F                  Wood fuel      0          MI-1   2.50   2.00   
                                                1          MI-1   2.50   2.00   
                                                2          MI-1   2.50   2.00   
                                                3          MI-1   2.50   2.00   
                                                4          MI-1   2.50   2.00   
...                                                                ...    ...   
Multi-family  B                  Electricity    4675       LC24   0.34   0.25   
                                                4676       LC24   0.34   0.25   
                                                4677       LC24   0.34   0.25   
                                                4678       LC24   0.34   0.25   
                                                4679       LC24   0.34   0.25   

                                                                  Roof  \
Housing type  Energy performance Heating energy r_position Class         
Single-family F                  Wood fuel      0          MI-1    2.5   
                                                1          MI-1    2.5   
                                                2          MI-1    2.5   
                                                3          MI-1    2.5   
                                                4          MI-1    2.5   
...                                                                ...   
Multi-family  B                  Electricity    4675       LC24    0.2   
                                                4676       LC24    0.2   
                                                4677       LC24    0.2   
                                                4678       LC24    0.2   
                                                4679       LC24    0.2   

                                                                  Windows  \
Housing type  Energy performance Heating energy r_position Class            
Single-family F                  Wood fuel      0          MI-1       2.4   
                                                1          MI-1       2.4   
                                                2          MI-1       2.4   
                                                3          MI-1       2.4   
                                                4          MI-1       2.4   
...                                                                   ...   
Multi-family  B                  Electricity    4675       LC24       1.6   
                                                4676       LC24       1

In [133]:
# Checks
print("profeel building stock: {}".format(profeel['Stock buildings'].sum()))
print("sdes building stock: {}".format(sdes['Stock buildings'].sum()))
print("The number of new lines for each profeel archetypes: {}".format(merged.shape[0]/profeel.shape[0]))
print("merged building stock: {}".format(merged['Stock buildings_x'].sum()/(merged.shape[0]/profeel.shape[0])))
print("output building stock: {}".format(cleaned['Stock buildings'].sum()))
print("building stock loss: {}".format(profeel['Stock buildings'].sum() - cleaned['Stock buildings'].sum()))

profeel building stock: 35172600
sdes building stock: 26698690.691502385
The number of new lines for each profeel archetypes: 120.0
merged building stock: 35172600.0
output building stock: 35172600.0
building stock loss: 0.0


In [ ]:
path_output = os.path.join(os.getcwd(), 'preprocessing/building_stock/output')
cleaned.to_csv(os.path.join(path_output, 'building_stock_profeel_hhdata.csv'))

# To be deleted

In [63]:
occup_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy']).sum()
tenant_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()
owner_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner', 'Income tenant']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()

profeel.merge(sdes, how='outer', on=['Housing type', 'Energy performance', 'Heating energy'])

Heating system  \
Housing type  Energy performance Heating energy                              
Single-family F                  Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
                                 Wood fuel       Wood fuel-Standard boiler   
...                                                                    ...   
              G                  Oil fuel                              NaN   
                                 Oil fuel                              NaN   
                                 Oil fuel                              NaN   
                                 Oil fuel                              NaN   
                                 Oil fuel                              NaN   

                                                 Wall  Floor  Roof  Windows  \
Housing type  Energy performance Heating energy                               
Single-family F                  Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
                                 Wood fuel        2.5    2.0   2.5      2.4   
...                                               ...    ...   ...      ...   
              G                  Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   
                                 Oil fuel         NaN    NaN   NaN      NaN   

                                                 Ventilation  \
Housing type  Energy performance Heating energy                
Single-family F                  Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
                                 Wood fuel               0.4   
...                                                      ...   
              G                  Oil fuel                NaN   
                                 Oil fuel                NaN   
                                 Oil fuel                NaN   
                                 Oil fuel                NaN   
                                 Oil fuel                NaN   

                                                 Stock buildings_x  \
Housing type  Energy performance Heating energy                      
Single-family F                  Wood fuel                688400.0   
                                 Wood fuel                688400.0   
                                 Wood fuel                688400.0   
                                 Wood fuel                688400.0   
                                 Wood fuel                688400.0   
...                                                            ...   
              G                  Oil fuel                      NaN   
                                 Oil fuel                      NaN   
                                 Oil fuel                      NaN   
                                 Oil fuel                      NaN   
                                 Oil fuel                      NaN   

                                                 Efficiency  energy_primary  \
Housing type  Energy performance Heating energy                               
Single-family F                  Wood fuel             0.75      423.726894   
                                 Woo

In [99]:
occup_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy']).sum()
profeel = profeel.groupby(level=['Housing type', 'Energy performance', 'Heating energy'])['Stock buildings'].sum().rename('Stock buildings')
tenant_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()
owner_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner', 'Income tenant']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()
profeel_occup = occup_prop.multiply(profeel, axis=0).dropna()

In [108]:
profeel_occup

Stock buildings
Housing type  Energy performance Heating energy Occupancy status                 
Multi-family  B                  Electricity    Owner-occupied       7.558572e+05
                                                Privately rented     1.243852e+06
                                                Social-housing       6.412905e+05
              C                  Electricity    Owner-occupied       2.656255e+05
                                                Privately rented     3.689851e+05
                                                Social-housing       1.333895e+05
                                 Natural gas    Owner-occupied       2.565633e+05
                                                Privately rented     2.197016e+05
                                                Social-housing       3.600352e+05
              D                  Electricity    Owner-occupied       5.451666e+05
                                                Privately rented     9.907463e+05
                                                Social-housing       3.281870e+05
                                 Natural gas    Owner-occupied       6.136537e+05
                                                Privately rented     5.347581e+05
                                                Social-housing       7.070882e+05
              E                  Electricity    Owner-occupied       7.399848e+05
                                                Privately rented     1.664741e+06
                                                Social-housing       4.916743e+05
                                 Natural gas    Owner-occupied       7.785191e+05
                                                Privately rented     8.417635e+05
                                                Social-housing       6.101175e+05
              F                  Electricity    Owner-occupied       1.346884e+05
                                                Privately rented     3.943556e+05
                                                Social-housing       7.045594e+04
                                 Natural gas    Owner-occupied       4.927132e+05
                                                Privately rented     6.985192e+05
                                                Social-housing       3.875676e+05
              G                  Electricity    Owner-occupied       1.225168e+05
                                                Privately rented     4.738726e+05
                                                Social-housing       3.641059e+04
Single-family B                  Natural gas    Owner-occupied       1.507268e+06
                                                Privately rented     2.417782e+05
                                                Social-housing       3.701543e+05
              C                  Natural gas    Owner-occupied       7.958925e+05
                                                Privately rented     1.214964e+05
                                                Social-housing       1.359111e+05
              D                  Electricity    Owner-occupied       1.439418e+06
                                                Privately rented     3.289199e+05
                                                Social-housing       6.766259e+04
                                 Natural gas    Owner-occupied       4.801252e+05
                                                Privately rented     6.682596e+04
                                                Social-housing       5.664886e+04
              E                  Electricity    Owner-occupied       2.565609e+06
                                                Privately rented     5.837361e+05
                                                Social-housing       2.230552e+05
                                 Natural gas    Owner-occupied       3.594915e+05
                                                Privately rented     5.301836e+04
                                                

In [111]:
owner_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()/sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status']).sum()
profeel_occup_owner = owner_prop.multiply(profeel_occup, axis=0).dropna()

In [112]:
profeel_occup_owner

Stock buildings
Housing type  Energy performance Heating energy Occupancy status Income owner                 
Multi-family  B                  Electricity    Owner-occupied   D1               15648.149523
                                                                 D10             145604.613178
                                                                 D2               30312.341933
                                                                 D3               39656.950882
                                                                 D4               44546.079668
...                                                                                        ...
Single-family G                  Wood fuel      Privately rented D6                6807.165786
                                                                 D7                8264.703866
                                                                 D8               10634.455001
                                                                 D9               14262.287552
                                                Social-housing   D10               4592.889676

[420 rows x 1 columns]

In [113]:
tenant_prop = sdes.groupby(level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status',
                                 'Income owner', 'Income tenant']).sum() / sdes.groupby(
    level=['Housing type', 'Energy performance', 'Heating energy', 'Occupancy status', 'Income owner']).sum()
profeel_occup_owner_tenant = tenant_prop.multiply(profeel_occup_owner, axis=0).dropna()

In [114]:
profeel_occup_owner_tenant

Stock buildings
Housing type  Energy performance Heating energy Occupancy status Income owner Income tenant                 
Multi-family  B                  Electricity    Owner-occupied   D1           D1                15648.149523
                                                                 D10          D10              145604.613178
                                                                 D2           D2                30312.341933
                                                                 D3           D3                39656.950882
                                                                 D4           D4                44546.079668
...                                                                                                      ...
Single-family G                  Wood fuel      Social-housing   D10          D5                  455.088785
                                                                              D6                  351.906208
                                                                              D7                  260.714629
                                                                              D8                  168.934239
                                                                              D9                   90.464177

[2400 rows x 1 columns]